<a href="https://colab.research.google.com/github/vlad98rus/-netology_pyda/blob/HomeWork-1/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание "Введение и классификация рекомендательных систем"

Постройте топ фильмов в категориях Action и Comedy (или любых других).

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [0]:
movies = pd.read_csv('movies.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
ratings = pd.read_csv('ratings.csv')

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
# соединяем таблицу с рейтингами и названиями фильмов
joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

In [18]:
'Adventure' in 'Adventure|Animation|Children|Comedy|Fantasy'

True

In [0]:
# поставим метки фильмам Action и Comedy
joined_ratings['IsAction'] = joined_ratings['genres'].apply(lambda x: 'Action' in x)
joined_ratings['IsComedy'] = joined_ratings['genres'].apply(lambda x: 'Comedy' in x)

In [20]:
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,IsAction,IsComedy
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,False,True
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,False,True
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,True,False
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,False,False
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,False,False


### Находим пользователей (с большим количеством просмотров) и сортируем фильмы по  рейтингам этих пользователей: средняя оценка фильма, умноженная на нормированное количество рейтингов

In [0]:
# достанем топ 100 пользователей по количеству оценок
top_users = joined_ratings.groupby('userId')[['rating']].count().sort_values('rating', ascending=False)
top_users = top_users.reset_index().head(100)['userId'].values

In [23]:
# наши пользователи
top_users

array([414, 599, 474, 448, 274, 610,  68, 380, 606, 288, 249, 387, 182,
       307, 603, 298, 177, 318, 232, 480, 608, 600, 483, 590, 105,  19,
       305, 489, 111, 438, 217, 140, 477, 555,  91,  28, 219, 534,  89,
        64, 226, 561,  18, 525,  57, 381, 368, 509, 469, 560, 462, 292,
        21, 597,  42, 294, 160, 580, 596, 202, 275, 517,  45, 156, 514,
       391, 567, 357, 103, 339,  62, 199, 125,  51, 132,  66, 313, 200,
       221,   6, 453,  50, 425, 428, 573, 352,  84, 122, 382, 356, 239,
       135, 365, 484, 104,  63, 325, 169, 332, 290])

In [0]:
# производим фильтрация наших пользователей по метрике Action
joined_ratings = joined_ratings[joined_ratings['userId'].isin(top_users) & (joined_ratings['IsAction']==True)]

In [25]:
# достанем по каждому фильму количество рейтингов
title_num_ratings = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_num_ratings[title] = group.userId.unique().shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
# достанем простые статистики по количеству рейтингов
min_num_ratings = np.min([title_num_ratings[f] for f in title_num_ratings.keys()])
max_num_ratings = np.max([title_num_ratings[f] for f in title_num_ratings.keys()])
mean_num_ratings = np.mean([title_num_ratings[f] for f in title_num_ratings.keys()])

In [27]:
# считаем средний рейтинг на каждый фильм
title_mean_rating = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_mean_rating[title] = group.rating.mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
film_with_our_mark = []

# посчитаем нашу метрику для каждого фильма из датасета
for f in title_num_ratings.keys():
    film_with_our_mark.append(
        (f, title_mean_rating[f] * (title_num_ratings[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings))
    )

In [31]:
# выводим топ 10
list(sorted(film_with_our_mark, key=lambda x: x[1], reverse=True))[:10]

[('Star Wars: Episode IV - A New Hope (1977)', 3.7392392799257337),
 ('Matrix, The (1999)', 3.692464925396876),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 3.6499456409341797),
 ('Fight Club (1999)', 3.3971974594143988),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  3.302876761451928),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 3.301060646531009),
 ('Saving Private Ryan (1998)', 2.8664318064096266),
 ('Jurassic Park (1993)', 2.8635032473138278),
 ('Terminator 2: Judgment Day (1991)', 2.7327640019656543),
 ('Men in Black (a.k.a. MIB) (1997)', 2.6361042600500455)]

In [0]:
def get_top_films(df,
                  genre = 'Action',
                  top_users = 100,
                  top_films = 10
                 ):
    # поставим метки фильмам
    df['IsGenre'] = df['genres'].apply(lambda x: genre in x)
    # достанем топ пользователей по количеству оценок
    top_users_df = df.groupby('userId')[['rating']].count().sort_values('rating', ascending=False)
    top_users_list = top_users_df.reset_index()[:top_users]['userId'].values
    # отфильтруем только оценки наших пользователей и только фильмы Action
    df = df[df['userId'].isin(top_users_list) & (df['IsGenre']==True)]

    # достанем по каждому фильму количество рейтингов
    title_num_ratings = {}
    for title, group in tqdm_notebook(df.groupby('title')):
        title_num_ratings[title] = group.userId.unique().shape[0]

    # достанем простые статистики по количеству рейтингов
    min_num_ratings = np.min([title_num_ratings[f] for f in title_num_ratings.keys()])
    max_num_ratings = np.max([title_num_ratings[f] for f in title_num_ratings.keys()])
    mean_num_ratings = np.mean([title_num_ratings[f] for f in title_num_ratings.keys()])

    # считаем средний рейтинг на каждый фильм
    title_mean_rating = {}
    for title, group in tqdm_notebook(df.groupby('title')):
        title_mean_rating[title] = group.rating.mean()

    film_with_our_mark = []
    # посчитаем нашу метрику для каждого фильма из датасета
    for f in title_num_ratings.keys():
        film_with_our_mark.append(
            (f, title_mean_rating[f] * (title_num_ratings[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings))
        )

    # выводим топ 10 и получилось уже очень неплохо
    return list(sorted(film_with_our_mark, key=lambda x: x[1], reverse=True))[:top_films]

In [0]:
# соединим таблицу с рейтингами и названиями фильмов
joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

In [35]:
# Actions
get_top_films(joined_ratings, genre='Action', top_users=100, top_films=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[('Star Wars: Episode IV - A New Hope (1977)', 3.7392392799257337),
 ('Matrix, The (1999)', 3.692464925396876),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 3.6499456409341797),
 ('Fight Club (1999)', 3.3971974594143988),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  3.302876761451928),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 3.301060646531009),
 ('Saving Private Ryan (1998)', 2.8664318064096266),
 ('Jurassic Park (1993)', 2.8635032473138278),
 ('Terminator 2: Judgment Day (1991)', 2.7327640019656543),
 ('Men in Black (a.k.a. MIB) (1997)', 2.6361042600500455)]

In [37]:
# Comedy
get_top_films(joined_ratings, genre='Comedy', top_users=100, top_films=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[('Forrest Gump (1994)', 3.7831057251660605),
 ('Pulp Fiction (1994)', 3.6618024450292146),
 ('Back to the Future (1985)', 2.9482724292484126),
 ('Monty Python and the Holy Grail (1975)', 2.9089763045882226),
 ('Groundhog Day (1993)', 2.8724301308488047),
 ('Fargo (1996)', 2.8691194474477393),
 ('Toy Story (1995)', 2.7985254105010653),
 ('Truman Show, The (1998)', 2.6293613815502317),
 ('Men in Black (a.k.a. MIB) (1997)', 2.5871585243236908),
 ("Ferris Bueller's Day Off (1986)", 2.534442297060925)]